### Note: Possibility of improvements over the choices of p(t), a0, c0

This notebook aims to showcase Ising solvers running simulated bifurcation.

References:
1. https://advances.sciencemag.org/content/5/4/eaav2372
2. https://advances.sciencemag.org/content/7/6/eabe7953

In [8]:
import numpy as np
import scipy as sp
import numba as nb
import multiprocessing as mp
import time
import os
from scipy.sparse import csr_matrix

### Setting float precision

In [25]:
flt = np.float32

### SB algorithms

In [40]:
#@nb.njit(parallel=False)
def one_aSB_run(Q_matrix, steps, dt, Kerr_coef, a0, c0, init_y):
    """
    One (adiabatic) simulated bifurcation run over the full pump schedule.
    
    Parameters:
        Q_matrix (2-D array of float): The matrix representing the coupling field of the problem.
                                       Notice that the diagonal elements should be all zero.
        steps (int): The number of iterations.
        dt (float): Time step for the discretized time.
        Kerr_coef (float): The Kerr coefficient.
        a0, c0 (float): Positive constants.
        init_y (1-D array of float): Initial y.
    
    Return: final_state (1-D array of float)
    """
    
    #if np.diagonal(Q_matrix).any():
    #    raise ValueError("Diagonal elements of Q should be all zero.")
    
    #np.random.seed(sd)
    
    N = Q_matrix.shape[0]
    x = flt(np.zeros(N))
    y = flt(init_y)
    
    for k in range(steps):
        x += a0 * y * dt
        y -= (Kerr_coef * x**3 + a0 * (1. - k/steps) * x - c0 * Q_matrix.dot(x)) * dt
        #x_history[k] = x # for analysis purposes
    
    return np.sign(x)

In [41]:
#@nb.njit(parallel=False)
def one_bSB_run(Q_matrix, steps, dt, a0, c0, init_y):
    """
    One ballistic simulated bifurcation run over the full pump schedule.
    
    Parameters:
        Q_matrix (2-D array of float): The matrix representing the local and coupling field of the problem.
                                       Notice that the diagonal elements should be all zero.
        steps (int): The number of iterations.
        dt (float): Time step for the discretized time.
        a0, c0 (float): Positive constants.
        init_y (1-D array of float): Initial y.
    
    Return: final_state (1-D array of float)
    """
    
    #if np.diagonal(Q_matrix).any():
    #    raise ValueError("Diagonal elements of Q should be all zero.")
    
    #np.random.seed(sd)
    
    N = Q_matrix.shape[0]
    x = flt(np.zeros(N))
    y = flt(init_y)
    
    for k in range(steps):
        x += a0 * y * dt
        y -= (a0 * (1. - k/steps) * x - c0 * Q_matrix.dot(x)) * dt # pump increases from 0 to a0 linearly
        for i in nb.prange(N): # parallelizable
            if np.abs(x[i]) > 1:
                x[i] = np.sign(x[i])
                y[i] = 0
        #x_history[k] = x # for analysis purposes
    
    return np.sign(x)

In [42]:
#@nb.njit(parallel=False)
def one_dSB_run(Q_matrix, steps, dt, a0, c0, init_y):
    """
    One discrete simulated bifurcation run over the full pump schedule.
    
    Parameters:
        Q_matrix (2-D array of float): The matrix representing the local and coupling field of the problem.
                                       Notice that the diagonal elements should be all zero.
        steps (int): The number of iterations.
        dt (float): Time step for the discretized time.
        a0, c0 (float): Positive constants.
        init_y (1-D array of float): Initial y.
    
    Return: final_state (1-D array of float)
    """
    
    #if np.diagonal(Q_matrix).any():
    #    raise ValueError("Diagonal elements of Q should be all zero.")
    
    #np.random.seed(sd)
    
    N = Q_matrix.shape[0]
    x = flt(np.zeros(N))
    y = flt(init_y)
    
    for k in range(steps):
        x += a0 * y * dt
        y -= (a0 * (1. - k/steps) * x - c0 * Q_matrix.dot(np.sign(x))) * dt
        for i in nb.prange(N): # parallelizable
            if np.abs(x[i]) > 1:
                x[i] = np.sign(x[i])
                y[i] = 0
        #x_history[k] = x # for analysis purposes
    
    return np.sign(x)

### Import problem instances
Make sure diagonal elements are all zero. SB algorithms do not accept local fields.

In [44]:
def Ising_from_file(abs_file_path):
    with open(abs_file_path, 'r') as f:
        coef_lst = f.read().split()
    
    return csr_matrix(([int(x) for x in coef_lst[4::3]], ([int(x)-1 for x in coef_lst[2::3]], \
                        [int(x)-1 for x in coef_lst[3::3]])), shape=(int(coef_lst[0]), int(coef_lst[0])))

In [45]:
abs_file_path = os.getcwd() + f"/Gset/G11.txt" # absolute dir
J = Ising_from_file(abs_file_path)
n = 2000
dt = flt(200/n)
Kerr_coef = flt(1.)
a0 = flt(0.5)
c0 = flt(0.3)

np.random.seed(0)
init_y = np.random.uniform(flt(-0.1), flt(0.1), Q.shape[0])

In [43]:
# Without numba
x_history = np.zeros((n, 2))
start_time = time.time()
ans = one_aSB_run(J, n, dt, Kerr_coef, a0, c0, init_y)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

ground state: [ 1.  1.  1. -1. -1. -1. -1. -1. -1.  1. -1.  1.  1.  1. -1. -1. -1. -1.
  1.  1.  1. -1.  1.  1. -1. -1.  1.  1.  1.  1.  1. -1.  1. -1. -1.  1.
  1.  1. -1.  1.  1. -1. -1. -1.  1. -1.  1.  1. -1.  1.  1. -1.  1. -1.
  1.  1.  1. -1. -1.  1. -1.  1. -1. -1. -1.  1. -1. -1.  1.  1.  1. -1.
 -1.  1. -1.  1.  1. -1.  1.  1. -1. -1.  1.  1.  1. -1.  1. -1. -1.  1.
 -1.  1. -1.  1. -1. -1.  1. -1. -1. -1. -1.  1. -1.  1. -1.  1. -1.  1.
  1. -1. -1.  1. -1. -1. -1. -1. -1.  1. -1. -1. -1.  1.  1.  1.  1.  1.
  1.  1. -1.  1. -1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1. -1.  1. -1.
  1.  1. -1. -1.  1. -1.  1.  1.  1.  1. -1.  1.  1. -1. -1.  1. -1. -1.
  1.  1.  1. -1. -1.  1. -1.  1.  1. -1. -1. -1. -1.  1. -1. -1.  1.  1.
  1. -1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1. -1.  1.  1. -1.  1. -1.
  1.  1.  1.  1. -1. -1.  1.  1. -1. -1. -1.  1.  1. -1.  1.  1.  1. -1.
 -1.  1. -1. -1. -1. -1. -1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.
 -1. -1. -1. -1.  1.  1. -1.  1.  1. 

In [7]:
# Without numba
x_history = np.zeros((n, 2))
start_time = time.time()
ans = one_bSB_run(Q, n, dt, a0, c0)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

ground state: [-1. -1.]; time: 0.06502366065979004 s


In [10]:
# Without numba
x_history = np.zeros((n, 2))
start_time = time.time()
ans = one_dSB_run(Q, n, dt, a0, c0)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

ground state: [1. 1.]; time: 0.05405735969543457 s
